In [3]:
from sage.all import *
import multiprocessing
import json
import os
import gc
gc.enable()

# --- 前置函数 ---

def unicyc_gen(n):
    return graphs.nauty_geng(f"{n} {n}:{n} -c")

def graph_degseq_and_id(G):
    return (tuple(G.degree_sequence()), G.graph6_string())

def graph_id_girth_and_id(graph_id):
    G = Graph(graph_id)
    return (G.girth(), graph_id)

def graph_id_specrad_and_id(graph_id):
    specrad_tuple = Graph(graph_id).spectral_radius(prec=1e-6)
    specrad = int(specrad_tuple[0]*1e5)
    return (specrad, graph_id)

def graph_id_lapspec_and_id(graph_id):
    G = Graph(graph_id)
    lap_spectrum_tuple = tuple(G.spectrum(laplacian=True))
    return (lap_spectrum_tuple, graph_id)

def graph_id_sgnlap_charpoly(graph_id):
    sgnlap_mat = Graph(graph_id).laplacian_matrix(signless=True)
    sgnlap_charpoly = tuple(sgnlap_mat.characteristic_polynomial())
    return sgnlap_charpoly

# 删除字典中值为单个元素的键值对
def delete_single_value(dic):
    return {k: v for k, v in dic.items() if len(v) > 1}

# 返回字典或列表中所有值列表的长度和
def total_len(iter):
    if type(iter) == dict:
        return sum(len(value_list) for value_list in iter.values())
    else:
        return sum(len(elem) for elem in iter)

def process_batch_func(pool, graph_id_list, func, batch_size=1000):
    batch_list = []
    result_dic = {}
    for graph_id in graph_id_list:
        batch_list.append(graph_id)
        if len(batch_list) >= batch_size:
            results = pool.imap_unordered(func, batch_list)
            for func_result, graph_id in results:
                result_dic.setdefault(func_result, []).append(graph_id)
            batch_list = [] # 清空批处理列表
            del results
    if batch_list:  # 循环结束后处理剩余的图
        results = pool.imap_unordered(func, batch_list)
        for func_result, graph_id in results:
            result_dic.setdefault(func_result, []).append(graph_id)
        del results
    del batch_list
    gc.collect()

    result_list = [sublist for sublist in result_dic.values() if len(sublist)>1]
    return result_list

def lapspec_list(graph_id_list):
    if len(graph_id_list) == 2:
        lapspec_1 = Graph(graph_id_list[0]).spectrum(laplacian=True)
        lapspec_2 = Graph(graph_id_list[1]).spectrum(laplacian=True)
        if lapspec_1 == lapspec_2:
            return [graph_id_list]
        else:
            return []
    else:
        clsf_lapspec_dir = {}
        for graph_id in graph_id_list:
            lapspec = tuple(Graph(graph_id).spectrum(laplacian=True))
            clsf_lapspec_dir.setdefault(lapspec, []).append(graph_id)
        clsf_lapspec_list = [sublist for sublist in clsf_lapspec_dir.values() if len(sublist)>1]
        return clsf_lapspec_list

def sgnlap_charpoly_list(graph_id_list):
    if len(graph_id_list) == 2:
        if graph_id_sgnlap_charpoly(graph_id_list[0]) == graph_id_sgnlap_charpoly(graph_id_list[1]):
            return [graph_id_list]
        else:
            return []
    else:
        clsf_sgnlap_charpoly_dir = {}
        for graph_id in graph_id_list:
            sgnlap_charpoly = graph_id_sgnlap_charpoly(graph_id)
            clsf_sgnlap_charpoly_dir.setdefault(sgnlap_charpoly, []).append(graph_id)
        clsf_sgnlap_charpoly_list = [sublist for sublist in clsf_sgnlap_charpoly_dir.values() if len(sublist)>1]
        return clsf_sgnlap_charpoly_list

# --- 主函数 ---
def process_unicyc_graphs(n, batch_size=1000):
    gen = unicyc_gen(n)
    output_path = f"unicyc_graph_{n}"
    os.makedirs(output_path, exist_ok = True)
    # num_processes = int(6)
    pool = multiprocessing.Pool() # processes = num_processes

    # Step 1: 根据度数序列分类
    print("Step 1: Classifying by degree sequence.")
    clsf_degseq = {} # 分类字典, key为degseq, value为graph_id
    batch_list = [] # 批处理列表
    count_graph = 0
    for G in gen:
        batch_list.append(G)
        count_graph += 1
        if len(batch_list) >= batch_size: # 达到批处理大小
            print(f"\r Counted {count_graph} graphs.", end = ' ')
            # 并行计算图序列和标识,即graph_degseq_and_id对batch中每个图作用
            results = pool.imap_unordered(graph_degseq_and_id, batch_list)
            for degseq, graph_id in results:
                # 键值对 deg_seq : graph_id存入字典 clsf_degseq
                clsf_degseq.setdefault(degseq, []).append(graph_id)
            batch_list = [] # 清空批处理列表
    if batch_list:  # 循环结束后处理剩余的图
        results = pool.imap_unordered(graph_degseq_and_id, batch_list)
        for degseq, graph_id in results:
            clsf_degseq.setdefault(degseq, []).append(graph_id)
    del batch_list

    # 剔除单元素类并预处理
    clsf_degseq_list = [sublist for sublist in clsf_degseq.values() if len(sublist)>1]
    del clsf_degseq

    clsf_degseq_len, clsf_degseq_total_len = len(clsf_degseq_list), total_len(clsf_degseq_list)
    print(f"\n{clsf_degseq_len} classes and {clsf_degseq_total_len} graphs left after classifying by degree sequence.")

    # 输出到json中
    clsf_degseq_json = json.dumps(clsf_degseq_list)
    output_degseq = f"{output_path}/unicyc_{n}_degseq.json"
    with open(output_degseq,'w') as file:
        file.write(clsf_degseq_json)
    del clsf_degseq_json
    gc.collect()

    print(f"Successfully write in {output_degseq}")

    # Step 2: 根据围长分类
    print("Step 2: Classifying by girth.")
    clsf_girth_list = []
    count_list, count_graph = 0, 0
    for graph_id_list in clsf_degseq_list:
        count_list += 1
        count_graph += len(graph_id_list)
        print(f"\r Counted {count_list} lists ({float(count_list/clsf_degseq_len) :.2%}) and {count_graph} graphs ({float(count_graph/clsf_degseq_total_len) :.2%}), counting {len(graph_id_list)} graphs.", end = ' ')
        clsf_girth_sublist = process_batch_func(pool, graph_id_list, graph_id_girth_and_id, batch_size)
        clsf_girth_list.extend(clsf_girth_sublist)

    clsf_girth_len, clsf_girth_total_len = len(clsf_girth_list), total_len(clsf_girth_list)
    print(f"\n{clsf_girth_len} classes and {clsf_girth_total_len} graphs left after classifying by girth.")
    # 输出到json中
    clsf_girth_json = json.dumps(clsf_girth_list)
    output_girth = "{}/unicyc_{}_girth.json".format(output_path,n)
    with open(output_girth,'w') as file:
        file.write(clsf_girth_json)
    del clsf_girth_json
    gc.collect()
    print("Successfully write in {}".format(output_girth))

    # Step 3: 根据谱半径分类
    print("Step 3: Classifying by spectral radius.")
    clsf_specrad_list = []
    count_list, count_graph = 0, 0
    for graph_id_list in clsf_girth_list:
        count_list += 1
        count_graph += len(graph_id_list)
        print(f"\r Counted {count_list} lists ({float(count_list/clsf_girth_len) :.2%}) and {count_graph} graphs ({float(count_graph/clsf_girth_total_len) :.2%}), counting {len(graph_id_list)} graphs.", end = ' ')
        clsf_specrad_sublist = process_batch_func(pool, graph_id_list, graph_id_specrad_and_id, batch_size)
        clsf_specrad_list.extend(clsf_specrad_sublist)
    del clsf_degseq_list

    clsf_specrad_len, clsf_specrad_total_len = len(clsf_specrad_list), total_len(clsf_specrad_list)
    print(f"\n{clsf_specrad_len} classes and {clsf_specrad_total_len} graphs after classifying by spectral radius.")
    # 输出到json中
    clsf_specrad_json = json.dumps(clsf_specrad_list)
    output_specrad = "{}/unicyc_{}_specrad.json".format(output_path,n)
    with open(output_specrad,'w') as file:
        file.write(clsf_specrad_json)
    del clsf_specrad_json
    gc.collect()
    print("Successfully write in {}".format(output_specrad))

    # Step 4: 根据Laplacian谱分类
    print("Step 4: Classifying by Laplacian spectrum.")
    clsf_lapspec_list = []
    batch_list = []
    count_list, count_graph = 0, 0
    for graph_id_list in clsf_specrad_list:
        batch_list.append(graph_id_list)
        count_list += 1
        count_graph += len(graph_id_list)
        if len(batch_list) >= batch_size: # 达到批处理大小
            print(f"\r Counted {count_list} lists ({float(count_list/clsf_specrad_len) :.2%}) and {count_graph} graphs ({float(count_graph/clsf_specrad_total_len) :.2%}), counting {total_len(batch_list)} graphs.", end = ' ')
            clsf_lapspec_sublist = flatten(list(pool.imap_unordered(lapspec_list, batch_list)), max_level=1)
            clsf_lapspec_list.extend(clsf_lapspec_sublist)
            batch_list = [] # 清空批处理列表
            del clsf_lapspec_sublist
    if batch_list:  # 循环结束后处理剩余的图
        print(f"\r Counted {count_list} lists ({float(count_list/clsf_specrad_len) :.2%}) and {count_graph} graphs ({float(count_graph/clsf_specrad_total_len) :.2%}), counting {total_len(batch_list)} graphs.", end = '')
        clsf_lapspec_sublist = flatten(list(pool.imap_unordered(lapspec_list, batch_list)), max_level=1)
        clsf_lapspec_list.extend(clsf_lapspec_sublist)
        del clsf_lapspec_sublist
        del batch_list
    del clsf_specrad_list

    clsf_lapspec_len, clsf_lapspec_total_len = len(clsf_lapspec_list), total_len(clsf_lapspec_list)
    print(f"\n{clsf_lapspec_len} classes and {clsf_lapspec_total_len} graphs left after classifying by Laplacian spectrum.")
    # 输出到json中
    clsf_lapspec_json = json.dumps(clsf_lapspec_list)
    output_lapspec = "{}/unicyc_{}_lapspec.json".format(output_path,n)
    with open(output_lapspec,'w') as file:
        file.write(clsf_lapspec_json)
    del clsf_lapspec_json
    gc.collect()
    print("Successfully write in {}".format(output_lapspec))

    # Step 5: 根据无符号Laplace矩阵的特征多项式分类
    print("Step 5: Classifying by signless Laplacian charpoly.")
    clsf_sgnlap_charpoly_list = []
    batch_list = []
    count_list, count_graph = 0, 0
    for graph_id_list in clsf_lapspec_list:
        batch_list.append(graph_id_list)
        count_list += 1
        count_graph += len(graph_id_list)
        if len(batch_list) >= batch_size: # 达到批处理大小
            print(f"\r Counted {count_list} lists ({float(count_list/clsf_specrad_len) :.2%}) and {count_graph} graphs ({float(count_graph/clsf_specrad_total_len) :.2%}), counting {total_len(batch_list)} graphs.", end = ' ')
            clsf_sgnlap_charpoly_sublist = flatten(list(pool.imap_unordered(sgnlap_charpoly_list, batch_list)), max_level=1)
            clsf_sgnlap_charpoly_list.extend(clsf_sgnlap_charpoly_sublist)
            batch_list = [] # 清空批处理列表
            del clsf_sgnlap_charpoly_sublist
    if batch_list:  # 循环结束后处理剩余的图
        print(f"\r Counted {count_list} lists ({float(count_list/clsf_lapspec_len) :.2%}) and {count_graph} graphs ({float(count_graph/clsf_lapspec_total_len) :.2%}), counting {total_len(batch_list)} graphs.", end = '')
        clsf_sgnlap_charpoly_sublist = flatten(list(pool.imap_unordered(sgnlap_charpoly_list, batch_list)), max_level=1)
        clsf_sgnlap_charpoly_list.extend(clsf_sgnlap_charpoly_sublist)
        del clsf_sgnlap_charpoly_sublist
        del batch_list
    del clsf_lapspec_list

    clsf_sgnlap_charpoly_len, clsf_sgnlap_charpoly_total_len = len(clsf_sgnlap_charpoly_list), total_len(clsf_sgnlap_charpoly_list)
    print(f"\n{clsf_sgnlap_charpoly_len} classes and {clsf_sgnlap_charpoly_total_len} graphs left after classifying by Signless Laplacian charpoly.")
    # 输出到json中
    clsf_sgnlap_charpoly_json = json.dumps(clsf_sgnlap_charpoly_list)
    output_sgnlap_charpoly = f"{output_path}/unicyc_{n}_sgnlap_charpoly.json"
    with open(output_sgnlap_charpoly,'w') as file:
        file.write(clsf_sgnlap_charpoly_json)
    del clsf_sgnlap_charpoly_json
    gc.collect()
    print("Successfully write in {}".format(output_sgnlap_charpoly))

    pool.close()
    pool.join()

In [ ]:
# %load_ext line_profiler
# %lprun -f process_unicyc_graphs -T lprof_output.txt process_unicyc_graphs(10)
process_unicyc_graphs(20, batch_size=10000)

In [15]:
# 从json中读取图列表处理
import json
import multiprocessing

def graph_id_genadj_charpoly(graph_id, mu):
    G = Graph(graph_id)
    genadj = (1-mu) * G.adjacency_matrix() - mu * G.kirchhoff_matrix()
    genadj_charpoly = tuple(genadj.characteristic_polynomial())
    return genadj_charpoly

def genadj_charpoly_list(graph_id_list, mu=-2):
    if len(graph_id_list) == 2:
        if graph_id_genadj_charpoly(graph_id_list[0], mu) == graph_id_genadj_charpoly(graph_id_list[1], mu):
            return [graph_id_list]
        else:
            return []
    else:
        clsf_genadj_charpoly_dir = {}
        for graph_id in graph_id_list:
            genadj_charpoly = graph_id_genadj_charpoly(graph_id, mu)
            clsf_genadj_charpoly_dir.setdefault(genadj_charpoly, []).append(graph_id)
        clsf_genadj_charpoly_list = [sublist for sublist in clsf_genadj_charpoly_dir.values() if len(sublist)>1]
        return clsf_genadj_charpoly_list

input_path = "unicyc_20_genadj_charpoly.json"
with open(input_path, 'r', encoding = 'utf-8') as file:
    graph_list = json.load(file)
graph_list_len, graph_list_total_len = len(graph_list), total_len(graph_list)

pool = multiprocessing.Pool()
clsf_genadj_charpoly_list = []
batch_list = []
count_list, count_graph = 0, 0
for graph_id_list in graph_list:
    batch_list.append(graph_id_list)
    count_list += 1
    count_graph += len(graph_id_list)
    if len(batch_list) >= 1000: # 达到批处理大小
        print(f"\r Counted {count_list} lists ({float(count_list/graph_list_len) :.2%}) and {count_graph} graphs ({float(count_graph/graph_list_total_len) :.2%}), counting {total_len(batch_list)} graphs.", end = ' ')
        clsf_sublist = flatten(list(pool.imap_unordered(genadj_charpoly_list, batch_list)), max_level=1)
        clsf_genadj_charpoly_list.extend(clsf_sublist)
        batch_list = [] # 清空批处理列表
        del clsf_sublist
if batch_list:  # 循环结束后处理剩余的图
    print(f"\r Counted {count_list} lists ({float(count_list/graph_list_len) :.2%}) and {count_graph} graphs ({float(count_graph/graph_list_total_len) :.2%}), counting {total_len(batch_list)} graphs.", end = '')
    clsf_sublist = flatten(list(pool.imap_unordered(genadj_charpoly_list, batch_list)), max_level=1)
    clsf_genadj_charpoly_list.extend(clsf_sublist)
    del clsf_sublist
    del batch_list
del graph_list

clsf_genadj_charpoly_len, clsf_genadj_charpoly_total_len = len(clsf_genadj_charpoly_list), total_len(clsf_genadj_charpoly_list)
print(f"\n{clsf_genadj_charpoly_len} classes and {clsf_genadj_charpoly_total_len} graphs left after classifying by Signless Laplacian charpoly.")
# 输出到json中
clsf_genadj_charpoly_json = json.dumps(clsf_genadj_charpoly_list)
output_genadj_charpoly = "unicyc_20_genadj_charpoly.json"
with open(output_genadj_charpoly,'w') as file:
    file.write(clsf_genadj_charpoly_json)
del clsf_genadj_charpoly_json
gc.collect()
print("Successfully write in {}".format(output_genadj_charpoly))

 Counted 744 lists (100.00%) and 1488 graphs (100.00%), counting 1488 graphs.
744 classes and 1488 graphs left after classifying by Signless Laplacian charpoly.
Successfully write in unicyc_20_genadj_charpoly.json
